In [1]:
import pandas as pd
import numpy as np

In [9]:
#PTS solar data next is the important one, tracking data
zipcodes = pd.read_excel("Zip Code Community.xlsx", 'Villages to Muni with Zip')

zipcodes.applylambda x: '0'+str(x)
, columns=['Zip Code']
print(zipcodes)


f_ashp = pd.read_excel('ResidentialASHPProjectDatabase 11.4.2019.xlsx', 'Sheet1', skiprows=3) 
df_ashp = df_ashp.drop([0]) #remove first null row

df_pv = pd.read_excel('PVinPTSwebsite.xlsx')

# df = df.iloc[2:]
# Date Rebate Payment Approved by MassCEC	Site City/Town	Site Zip Code	Installer Company Name	Heating Fuel Being Replaced	Cooling Type Being Replaced	# of Outdoor Units	# of Indoor Units	 Capacity of Heat Pumps at 5°F 	Single- Head Heat Pump #1	Single- Head Heat Pump #2	Single- Head Heat Pump #3	Multi-Head Heat Pump #1	Multi-Head Heat Pump #2	Multi-Head Heat Pump #3	Total System Costs	Receiving an Income-Based Adder?	Rebate Amount 

SyntaxError: invalid syntax (<ipython-input-9-88e4a5ca5ee3>, line 4)

In [4]:
#testing
# print(df.iloc[0])
# print(df.head(3))
# print(df['Site Zip Code'])
# print(df.iloc[pd.np.r_[:1,1:]])
# print(df)
# print(df)

print(zipcodes.loc[(zipcodes['City'] == 'Amherst')]['Zip Code'])

1    1002.0
2    1003.0
3    1004.0
Name: Zip Code, dtype: float64


In [5]:
def formatZipCode(df, columnName):
    for x in df[columnName]:
        if len(str(x)) > 5:
            # print("old x ------- ", x)
            x = x[:5]
            # print('new x', x)
        elif len(str(x)) < 5:
            if len(str(x)) == 4:
                x = '0' + str(x)
                # print('new zip code', x)
            else:
                # print("error in data, please check or handle", x)
                cityName = df.iloc[df.loc[(df['Site Zip Code']==x)].index-1]['Site City/Town'] 
                    #index is one off, maybe skip that row
                print("city in q", cityName.values[0])
                x = (zipcodes.loc[(zipcodes['City'] == cityName.values[0])]['Zip Code'].values)
                if len(x) > 0:
                    x = '0' + str(x[0])
                    print("new x", x)
                else: 
                    print("more formatting issues", x)
                #flag row 1098, which is weird
                #all caps in the name

formatZipCode(df_ashp, 'Site Zip Code')



city in q Walpole
new x 02081.0
city in q WEST YARMOUTH
more formatting issues []


In [ ]:
#Data Fields for Final Dataset 
dataFields = ['Municipality', 'Zip Code', 'Tech', 'Year', 'Month', 'Quantity', 'Average Cost',
    'Total Cost', 'Total Rebates', 'Average Rebate', 'Count Income-Eligible']

df_final = pd.DataFrame(columns=dataFields)

print(df_final)

#to_csv